# Exploring and Cleaning the data

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('train_aug_in_kag_org.csv')

In [3]:
data.head(10)

,title,text,title_text,label
0,squirrel financial wellbeing,A Squirrel account makes it easy to stick to your budget and save. Start for free today!,squirrel financial wellbeing. A Squirrel account makes it easy to stick to your budget and save. Start for free today!,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
1,senior insurance marketing,"Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.","senior insurance marketing. Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
2,3i financial group inc.,"A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more","3i financial group inc.. A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
3,brownstone insurance,"Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.","brownstone insurance. Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
4,pacificwide lending,Learn more about Pacificwide Lending and our commercial & residential loan services in California.,pacificwide lending. Learn more about Pacificwide Lending and our commercial & residential loan services in California.,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
5,city auto finance,"City Auto Finance is the Southeast's most trusted used-car floor plan financing company with locations in Memphis, TN; Mobile, AL; Huntsville, AL and Murfreesboro, TN. We provide you with competitive rates and the capability to make payments online.","city auto finance. City Auto Finance is the Southeast's most trusted used-car floor plan financing company with locations in Memphis, TN; Mobile, AL; Huntsville, AL and Murfreesboro, TN. We provide you with competitive rates and the capability to make payments online.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
6,"veterans financial, inc.","VA Aid and Attendance benefit can help pay for assisted living, home care or nursing home. Veterans Financial Inc can advise you at no cost about benefits","veterans financial, inc.. VA Aid and Attendance benefit can help pay for assisted living, home care or nursing home. Veterans Financial Inc can advise you at no cost 

In [4]:
data.shape

(34953, 4)

In [5]:
data.isnull().sum()

title         42
text           0
title_text     0
label          0
dtype: int64

In [6]:
data.dropna(inplace=True)
data.drop(columns=['text'], inplace=True)

In [7]:
data.head()

,title,title_text,label
0,squirrel financial wellbeing,squirrel financial wellbeing. A Squirrel account makes it easy to stick to your budget and save. Start for free today!,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
1,senior insurance marketing,"senior insurance marketing. Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
2,3i financial group inc.,"3i financial group inc.. A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
3,brownstone insurance,"brownstone insurance. Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
4,pacificwide lending,pacificwide lending. Learn more about Pacificwide Lending and our commercial & residential loan services in California.,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing


# Preparing our data for modeling 

In [8]:
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# title
title_corpus = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', data.iloc[i, 0]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    title_corpus.append(row)

In [10]:
# text
text_corpus = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', data.iloc[i, 1]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    text_corpus.append(row)

_Our Bag of Words_

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
title_cv = CountVectorizer(ngram_range=(1, 2), analyzer="word", max_features=300)   
X_title = title_cv.fit_transform(title_corpus).toarray()

In [13]:
X_title.shape

(34911, 300)

In [14]:
text_cv = CountVectorizer(ngram_range=(1, 2), analyzer="word", max_features=1200)   
X_text = text_cv.fit_transform(text_corpus).toarray()

In [15]:
X_text.shape

(34911, 1200)

In [16]:
X = np.concatenate((X_title, X_text), axis=1)
y = data.iloc[:, 2].values

In [17]:
X.shape, y.shape

((34911, 1500), (34911,))

In [18]:
# LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [19]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

# Splitting the data

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [21]:
X_train.shape, X_val.shape

((27928, 1500), (6983, 1500))

# Modeling

In [22]:
from time import time 

from sklearn.naive_bayes import GaussianNB, ComplementNB, MultinomialNB

from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

In [36]:
models = [
    ('GaussianNB', GaussianNB()),
    ('ComplementNB', ComplementNB()), 
    ('MultinomialNB', MultinomialNB())
]

results = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision-MacroAVG', 'Precision-WeightedAVG', 
                                  'Recall-MacroAVG', 'Recall-WeightedAVG', 'F1_score-MacroAVG','F1_score-WeightedAVG', 'Time_taken'])
for model_name, model in models:
    start = time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    end = time()
    
    record = {}
    record['Model'] = model_name
    record['Accuracy'] = accuracy_score(y_val, y_pred)*100
    record['Precision-MacroAVG'] = precision_score(y_val, y_pred, average='macro')*100
    record['Precision-WeightedAVG'] = precision_score(y_val, y_pred, average='weighted')*100
    record['Recall-MacroAVG'] = recall_score(y_val, y_pred, average='macro')*100
    record['Recall-WeightedAVG'] = recall_score(y_val, y_pred, average='weighted')*100
    record['F1_score-MacroAVG'] = f1_score(y_val, y_pred, average='macro')*100
    record['F1_score-WeightedAVG'] = f1_score(y_val, y_pred, average='weighted')*100
    record['Time_taken'] = end-start
    
    results = results.append(record, ignore_index=True)
    
results = results.sort_values(by='Accuracy', ascending=False)

In [37]:
results

,Model,Accuracy,Precision-MacroAVG,Precision-WeightedAVG,Recall-MacroAVG,Recall-WeightedAVG,F1_score-MacroAVG,F1_score-WeightedAVG,Time_taken
1,ComplementNB,83.846484,67.305035,82.729130,60.638036,83.846484,61.279612,82.406499,4.514647
2,MultinomialNB,83.703279,68.211160,85.579353,77.247976,83.703279,71.156935,84.376142,5.329692
0,GaussianNB,57.754547,49.829791,75.212289,60.661665,57.754547,48.437445,61.163903,1.377748


In [39]:
# saving the models
import joblib

joblib.dump(models[0][1], 'GaussianNB.sav')
joblib.dump(models[1][1], 'ComplementNB.sav')
joblib.dump(models[2][1], 'MultinomialNB.sav')

['MultinomialNB.sav']

# Testing on test_data

In [40]:
test_data = pd.read_csv('test_in_kag_org.csv')

In [41]:
test_data.shape

(6501, 4)

In [42]:
test_data.isnull().sum()

title         0
text          0
title_text    0
label         0
dtype: int64

In [43]:
test_data['label'].nunique()

15

In [44]:
test_data.drop(columns=['text'], inplace=True)

# Preparing our data for modeling 

In [45]:
# title
title_corpus_test = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(test_data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', test_data.iloc[i, 0]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    title_corpus_test.append(row)

In [46]:
# text
text_corpus_test = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(test_data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', test_data.iloc[i, 1]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    text_corpus_test.append(row)

In [47]:
len(text_corpus_test)

6501

In [48]:
text_corpus_test[0]

'centralized hub procore offer ability communicate team get full view project one place assign task stay connected google map google doc'

_Our Bag of Words_

In [49]:
X_title_test = title_cv.transform(title_corpus_test).toarray()

In [50]:
X_title_test.shape

(6501, 300)

In [51]:
X_text_test = text_cv.transform(text_corpus_test).toarray()

In [52]:
X_text_test.shape

(6501, 1200)

In [53]:
X_test = np.concatenate((X_title_test, X_text_test), axis=1)
y_test = test_data.iloc[:, 2].values

In [54]:
X_test.shape, y_test.shape

((6501, 1500), (6501,))

In [55]:
y_test = le.transform(y_test)

In [56]:
np.unique(y_test)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [58]:
GaussianNB_model    = joblib.load('GaussianNB.sav')
ComplementNB_model  = joblib.load('ComplementNB.sav')
MultinomialNB_model  = joblib.load('MultinomialNB.sav')

In [60]:
models = [
    ('GaussianNB', GaussianNB_model),
    ('ComplementNB', ComplementNB_model), 
    ('MultinomialNB', MultinomialNB_model)
]

results = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision-MacroAVG', 'Precision-WeightedAVG', 
                                  'Recall-MacroAVG', 'Recall-WeightedAVG', 'F1_score-MacroAVG','F1_score-WeightedAVG'])
for model_name, model in models:
    y_pred = model.predict(X_test)
    
    record = {}
    record['Model'] = model_name
    record['Accuracy'] = accuracy_score(y_test, y_pred)*100
    record['Precision-MacroAVG'] = precision_score(y_test, y_pred, average='macro')*100
    record['Precision-WeightedAVG'] = precision_score(y_test, y_pred, average='weighted')*100
    record['Recall-MacroAVG'] = recall_score(y_test, y_pred, average='macro')*100
    record['Recall-WeightedAVG'] = recall_score(y_test, y_pred, average='weighted')*100
    record['F1_score-MacroAVG'] = f1_score(y_test, y_pred, average='macro')*100
    record['F1_score-WeightedAVG'] = f1_score(y_test, y_pred, average='weighted')*100
    
    results = results.append(record, ignore_index=True)
    
results = results.sort_values(by='Accuracy', ascending=False)
results

,Model,Accuracy,Precision-MacroAVG,Precision-WeightedAVG,Recall-MacroAVG,Recall-WeightedAVG,F1_score-MacroAVG,F1_score-WeightedAVG
1,ComplementNB,89.478542,63.610026,89.112706,62.686290,89.478542,60.855531,88.919215
2,MultinomialNB,86.279034,57.170145,89.853797,73.910764,86.279034,62.595837,87.638338
0,GaussianNB,54.637748,34.676613,83.907349,46.489703,54.637748,32.164753,62.549027
